In [1]:
import pandas as pd
from glob import iglob
from itertools import chain
from sqlalchemy import create_engine

In [2]:
mta_engine = create_engine('sqlite:///mta.db')

for data_file in iglob('./data/*.txt'):
    try:
        with open(data_file) as data:
            df = pd.read_csv(data)
            #remove trailing whitespace from the column names
            df.columns = [c.strip().lower() for c in df.columns]
            #just use 'regular' records
            df = df[df.desc == 'REGULAR']
            #create proper timestamp column
            df['ts'] = pd.to_datetime(df.date + ' ' + df.time)
            #test for uniquness
            sum(df[['ts','c/a','unit', 'scp', 'station']].duplicated()) == 0
            #subset to minimum required set of vars
            df = df.drop(['date', 'time', 'linename', 'division', 'desc'], axis=1)
            #order by tunrstile identity and timeseries
            df = df.sort(['station','c/a','unit','scp','ts'])
            df = df.set_index(['station','c/a','unit','scp','ts'])
            #get the delta between observations and remove outliers and missing values
            gb = df.groupby(level=['station','c/a','unit','scp']).diff()
            gb = gb.fillna(-1)
            entries = gb[gb.entries >= 0]['entries']
            exits = gb[gb.exits >= 0]['exits']
            #only keep observations less than or equal to the 99% quantile
            #TODO: consider removing outliers at the station level, instead of across the board
            entries = entries[entries <= entries.quantile(.99)]
            exits = exits[exits <= exits.quantile(.99)]
            #create a clean dataframe
            df_clean = pd.DataFrame([entries.groupby(level=['station', 'ts']).sum(),exits.groupby(level=['station', 'ts']).sum()]).T
            #aggregate to the 'day' level instead of hourly
            df_clean = df_clean.reset_index()
            df_clean['ts'] = df_clean['ts'].apply(lambda x: x.date())
            df_clean = df_clean.rename(columns={'ts': 'day'})
            df_clean = df_clean.groupby(['station', 'day']).sum()
            df_clean.to_sql(name='data', con=mta_engine, if_exists='append')
    except Exception as e:
        with open('error.log', 'a') as errors:
            errors.write(data_file)
            errors.write(str(e))
            errors.write('\n')

/Users/dgabrieli/anaconda3/envs/py34/lib/python3.4/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/dgabrieli/anaconda3/envs/py34/lib/python3.4/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (7,12,17,22,27,32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/dgabrieli/anaconda3/envs/py34/lib/python3.4/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/dgabrieli/anaconda3/envs/py34/lib/python3.4/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (7,12,17,22,27,32) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
